In [1]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.image as mpimg
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from scipy import ndimage as ndi
import glob
from os import path 
import pandas as pd 


In [2]:
px_per_cm2 = 55800.1246202
min_size = 5000
topcrop = 200
botcrop = 10 
lcrop = 10
rcrop = 100

In [3]:
def find_leaves( original, min_size): 
    
    gray = rgb2gray(original)
    thresh = threshold_otsu(gray)
    binary = gray < thresh
    test, _ = ndi.label(binary)
    objs, count = np.unique(test, return_counts= True)
    large_index = count > min_size  
    objs = objs[large_index]
    count = count[large_index]
    objs = objs[np.argsort( count)[::-1 ]]
    leaf_index = objs[1:] # cut background object 
    leaf = np.isin( test, leaf_index )
 
    return( leaf )
    
def custom_crop(original, topcrop, botcrop, lcrop, rcrop ):
    
    or_shape = np.int_( original.shape)
    cropped = original[ topcrop:(or_shape[0] - botcrop), lcrop:(or_shape[1]-rcrop)]
    
    return(cropped)       

def plot_leaf_area(file_name, binary_img, leaf_area):
    
    fig, axes = plt.subplots(1, 2, figsize=(10,10))
    ax = axes.ravel()
    ax[0].imshow(mpimg.imread(file_name), cmap = plt.cm.gray)
    ax[0].set_title(file_name)
    ax[1].imshow(binary_img, cmap=plt.cm.binary)
    ax[1].set_title( "Leaf area: {:.2f} cm\N{SUPERSCRIPT TWO}".format(leaf_area)  )
    fig.tight_layout()
    plt.savefig( path.splitext( path.basename( file_name))[0] + "_results.jpg", dpi = 150)
    plt.close()


In [4]:
img_files = glob.glob("STME_*.jpeg")
img_files.sort()

In [5]:
leaf_area = list()
binary_imgs = list()

In [6]:
for i,f in enumerate( img_files ) : 
    img = mpimg.imread(f)
    cropped = custom_crop(img, topcrop, botcrop, lcrop, rcrop)
    lf = clear_border( find_leaves(cropped, min_size) )
    binary_imgs.append(lf)
    val, counts = np.unique(lf, return_counts = True)
    leaf_area.append( counts[1]/px_per_cm2 )
    plot_leaf_area(f, binary_imgs[i], leaf_area[i])

In [7]:
d = {'Slice': [path.splitext(f)[0] for f in img_files]}
my_dat = pd.DataFrame(data = d)
my_dat['Count'] = ''
my_dat['Total Area'] = leaf_area
my_dat['Average Size'] = ''
my_dat['%Area'] = ''
my_dat['Mean'] = ''

In [8]:
my_dat.to_csv('STME_leaf_area.csv', index= False)